In [1]:
import os
import numpy as np
import pandas as pd

import sys
sys.path.append('/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/algorithms')
# from ce_utils.record import printProgress, Sec_to_M_S_ms
# from ce_utils.video import load_frame, find_clip_idxs, merging, captioning
# from ce_utils.data import batch_idxs, Reshape4torch
# from ce_model.CNN_inference import CNN_model
from ce_utils.video import load_frames, model_pred, find_clip_idxs, get_gradcam, process_and_save
from ce_utils.record import sec_to_m_s_ms, total_sum_time
from ce_model.cnn_inf import cnn_model

In [7]:
gpu_idx = 0

model_dir = '/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/model_developlment/2_AI_feedback/4_add_200917/re-learning_200713_2_model/model/'

model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                              model_dir = model_dir, model_name = 'np_binary', gpu_idx = gpu_idx)

model:
np_binary_0.0001_16_2010162056_010_t_accr_0.9627_t_loss_0.349785_v_accr_0.9553_v_loss_0.357244.pt



In [11]:
frame_root = '/mnt/disk2/data/private_data/SMhospital/capsule/0 data/video/200204 clip for clinical test'
video_log = pd.read_csv(frame_root + '/readme.txt', header = None, delimiter = '|', encoding = "cp949")

In [12]:
video_log

,0,1,2
0,1,김 경숙 (03319309) 05 Aug 19_1.mpg,02:07.500
1,2,나 기용 (17816122) 29 Jul 19_1.mpg,01:16.500
2,3,모 금녀 (16548422) 10 Aug 18_1.mpg,06:19.500
3,4,박 완웅 (11123900) 05 Oct 18_1.mpg,01:19.500
4,5,송 복순 (08952031) 20 Dec 17_1.mpg,02:47.500
5,6,시로츠카 아키요 (20174705) 31 Jul 19_1.mpg,02:44.500
6,7,신 용철 (03526121) 17 Aug 18_1.mpg,07:21.500
7,8,이 송준 (12548633) 25 Jun 19_1.mpg,07:44.500
8,9,이 순천 (07422665) 18 Mar 19_1.mpg,04:17.500
9,10,이 재일 (05629743) 07 Feb 19_1.mpg,04:49.500


In [13]:
# 최종 처리 이후 csv 파일에 들어갈 column 이름들 (for statistical analysis)
record = pd.DataFrame(columns = ['index', 'video', '# of frames (a)', 'video length', 'save frame time', 'load frame time', 
                                 'pred. time', '# of positive frames (b)', 'screening rate (b/a)', '# of clip frames', 'clip length',
                                 'Grad-CAM time','process and save time', 'total computing time'])
record

,index,video,# of frames (a),video length,save frame time,load frame time,pred. time,# of positive frames (b),screening rate (b/a),# of clip frames,clip length,Grad-CAM time,process and save time,total computing time


In [16]:
print(video_log, '\n')
print('load frames...\n')

video_frames = []

fps = 5
for i in range(len(video_log)):
    
    video_info = video_log.iloc[i]
        
    idx, video_name, frame_save_time = (video_info[0],  video_info[1], video_info[2])
    print(idx, ':', video_name, '\n')

    frame_dir = os.path.join(frame_root, '{:02d}'.format(idx))

    # 프레임을 numpy 형태로 불러옴 & 걸린 시간도 가져옴
    frames_pre, frame_load_time = load_frames(frame_dir)
    print('load time:', frame_load_time, '\n')
    
    n_frame = len(frames_pre)
    
    print('model inference, processing, and saving... \n')

    # Binary Model과 이미지 파일을 model_pred_and_time 함수에 넣고 리턴으로 binary model의 예측 결과와 걸린 시간을 가져옴
    # 예측결과는 model의 softmax 결과 값이 담긴 list
    preds, pred_time = model_pred(model, frames_pre)
  
    # 각각의 예측 모델이 예측한 frame은 softmax의 argmax 결과가 1인 index에 해당하는 frame
    # prediction에 해당하는 index만 가져옴 
    pred_idx = np.where(np.argmax(preds, axis = 1) == 1)[0]
    screening_rate = '{:0.2f} %'.format(len(pred_idx)/n_frame*100)
    
    # prediction index 전후 5프레임씩 더하여, 비디오 클립에 필요한 index를 가져옴
    clip_idx = find_clip_idxs(pred_idx, len(frames_pre))

    # 모델과, 프레임 그리고 grad-cam이 필요한 프레임의 index를 통해 grad cam image를 구함
    cams, cam_time = get_gradcam(model, frames_pre, clip_idx, batch_size = 64, target_layers = ['conv7_2'], target_class = None)

    save_dir = os.path.join('./', '{:02d}'.format(idx))
    # 원본 frame과 grad-cam frame를 붙여서 아래 위치에 저장함
    processing_time = process_and_save(frame_dir, save_dir, frames_pre, cams, preds, clip_idx)

    total_time = total_sum_time([frame_save_time, frame_load_time, pred_time, cam_time, processing_time])  
        
    # 파일 저장 과정에서 단계별로  소요된 시간 계산
    record.loc[i] = [idx, video_name, n_frame, sec_to_m_s_ms(n_frame/fps), frame_save_time, frame_load_time, 
                     pred_time, len(pred_idx), screening_rate, len(clip_idx), sec_to_m_s_ms(len(clip_idx)/fps),
                     cam_time, processing_time, total_time]
    
    print()
    
record.to_csv('record.csv',encoding='utf-8-sig', index = False)

     0                                    1           2
0    1      김 경숙 (03319309) 05 Aug 19_1.mpg  02:07.500 
1    2      나 기용 (17816122) 29 Jul 19_1.mpg   01:16.500
2    3      모 금녀 (16548422) 10 Aug 18_1.mpg   06:19.500
3    4      박 완웅 (11123900) 05 Oct 18_1.mpg   01:19.500
4    5      송 복순 (08952031) 20 Dec 17_1.mpg   02:47.500
5    6  시로츠카 아키요 (20174705) 31 Jul 19_1.mpg   02:44.500
6    7      신 용철 (03526121) 17 Aug 18_1.mpg   07:21.500
7    8      이 송준 (12548633) 25 Jun 19_1.mpg   07:44.500
8    9      이 순천 (07422665) 18 Mar 19_1.mpg   04:17.500
9   10      이 재일 (05629743) 07 Feb 19_1.mpg   04:49.500
10  11      정 병국 (09609741) 27 Aug 18_1.mpg   01:57.500
11  12      정 선자 (13757465) 29 Jun 18_1.mpg   03:16.500
12  13      차 난희 (07808670) 03 Jul 18_1.mpg   02:19.500
13  14      최 금주 (07481795) 08 May 18_1.mpg   04:48.500
14  15      한 봉이 (20009008) 01 Jul 19_1.mpg   01:48.500 

load frames...

1 : 김 경숙 (03319309) 05 Aug 19_1.mpg 

6358 |##########################################

21010 |################################################################################ | 100.0% loading frames
load time: 04:20.890 

model inference, processing, and saving... 

21010: 1314(*16) |################################################## | 100.0% prediction
4171: 66(*64) |################################################################################ | 100.0% get gradcam in clip frame
4171 |################################################################################ | 100.0% process and save --> ./14
15 : 한 봉이 (20009008) 01 Jul 19_1.mpg 

7551 |################################################################################ | 100.0% loading frames
load time: 01:35.414 

model inference, processing, and saving... 

7551: 472(*16) |################################################## | 100.0% prediction
1868: 30(*64) |################################################################################ | 100.0% get gradcam in clip frame
1868 |####################################

In [ ]:
    for i in range(len(video_log)):
        video_info = video_log.iloc[i]

        idx, video_name, frame_save_time, n_frame = (video_info['index'], 
                                                     video_info['video'], 
                                                     video_info['elapsed time'], 
                                                     video_info['number of frames'])

        print('')
        print(idx, ':', video_name, n_frame, '\n')

        frame_dir = os.path.join(args.frame_root, '{:02d}'.format(idx))

        # 프레임을 numpy 형태로 불러옴 & 걸린 시간도 가져옴
        frames_pre, frame_load_time = load_frames(frame_dir)

        # Binary Model과 이미지 파일을 model_pred_and_time 함수에 넣고 리턴으로 binary model의 예측 결과와 걸린 시간을 가져옴
        # 예측결과는 model의 softmax 결과 값이 담긴 list
        preds, pred_time = model_pred(model, frames_pre)

        # 각각의 예측 모델이 예측한 frame은 softmax의 argmax 결과가 1인 index에 해당하는 frame
        # prediction에 해당하는 index만 가져옴 
        pred_idx = np.where(np.argmax(preds, axis = 1) == 1)[0]
        # prediction index 전후 5프레임씩 더하여, 비디오 클립에 필요한 index를 가져옴
        clip_idx = find_clip_idxs(pred_idx, len(frames_pre))

        # 모델과, 프레임 그리고 grad-cam이 필요한 프레임의 index를 통해 grad cam image를 구함
        cams, cam_time = get_gradcam(model, frames_pre, clip_idx, batch_size = 64, target_layers = ['conv7_2'], target_class = None)


        save_dir = os.path.join(args.save_root, '{:02d}'.format(idx))
        # 원본 frame과 grad-cam frame를 붙여서 아래 위치에 저장함
        processing_time = process_and_save(frame_dir, save_dir, frames_pre, cams, preds, clip_idx)

        total_time = total_sum_time([frame_save_time, frame_load_time, pred_time, cam_time, processing_time])

        # 파일 저장 과정에서 단계별로  소요된 시간 계산
        record.loc[i] = [idx, video_name, n_frame, Sec_to_M_S_ms(n_frame/fps), frame_save_time, frame_load_time, 
                         pred_time, len(pred_idx), len(clip_idx), Sec_to_M_S_ms(len(clip_idx)/fps),
                         cam_time, processing_time, total_time]


    record.to_csv(args.save_root + '/record.csv',encoding='utf-8-sig', index = False)
    